In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
import glob
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [119]:
df_hist = pd.read_csv('../input/historical_transactions.csv')

df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)

df_hist['installments'] = df_hist['installments'].map(lambda x:  
                                                    1 if x<1 else
                                                    1 if x>100 else
                                                      x
                                                     )
df_hist['authorized_flag'] = df_hist['authorized_flag'].map({'Y':1, 'N':0})
df_hist['category_1'] = df_hist['category_1'].map({'Y':1, 'N':0}) 
df_hist['yyyymm'] = df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_hist['yyyymmdd'] = df_hist['purchase_date'].map(lambda x: str(x)[:10])

auth1 = df_hist[df_hist['authorized_flag']==1]
auth0 = df_hist[df_hist['authorized_flag']==0]

In [44]:
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new,yyyymm,yyyymmdd
0,1,C_ID_4e6213e9bc,88,0,1,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,29.00,2017-06,2017-06-25
1,1,C_ID_4e6213e9bc,88,0,1,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,9.17,2017-07,2017-07-15
2,1,C_ID_4e6213e9bc,88,0,1,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,17.65,2017-08,2017-08-09
3,1,C_ID_4e6213e9bc,88,0,1,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,7.69,2017-09,2017-09-02
4,1,C_ID_4e6213e9bc,88,0,1,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,16.00,2017-03,2017-03-10


In [172]:
df = [auth1, auth0][1]
fname = ['auth1', 'auth0'][1]

df_id = df[key].drop_duplicates().to_frame().set_index(key)
month_list = pd.Series(np.sort(df['month_lag'].unique())).to_frame().T
df_id[month_list.columns] = month_list
df_id.iloc[:, :] = month_list.values
df_id.columns = month_list.values[0]
df_id = df_id.stack()
df_id.name = 'tmp'

month_min = df[[key, 'month_lag']].groupby(key).min()
df_id = df_id.to_frame().join(month_min)
df_id['last'] = df_id['tmp']>=df_id['month_lag']
df_id = df_id[df_id['last']]['tmp']
df_id.head()

card_id             
C_ID_4e6213e9bc  -11   -11.0
                 -10   -10.0
                 -9     -9.0
                 -8     -8.0
                 -7     -7.0
Name: tmp, dtype: float64

In [173]:
roll_sum = 1
aggs = {}

if roll_sum:
    aggs['purchase_amount_new'] = ['sum']
    aggs['authorized_flag'] = ['sum']
    aggs['installments'] = ['sum']
    aggs['purchase_date'] = ['count']
    aggs['category_1'] = ['sum']
    
else:
    aggs['purchase_amount_new'] = ['sum', 'mean', 'max', 'min', 'std']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['installments'] = ['mean', 'max', 'min']
    # aggs['yyyymm'] = ['nunique']
    aggs['yyyymmdd'] = ['nunique']
    aggs['purchase_date'] = ['count']
    aggs['subsector_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['merchant_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['city_id'] = ['nunique']
    aggs['category_1'] = ['sum', 'mean']

df_agg = df.groupby([key, 'month_lag']).agg(aggs)

def get_new_columns(aggs):
#     return [f'{fname}_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    return [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

new_cols = get_new_columns(aggs)
df_agg.columns = new_cols

In [174]:
df_id = df_id.to_frame().reset_index()
df_id.columns = [key, 'month_lag', 'tmp']
df_id.set_index([key, 'month_lag'], inplace=True)
df = df_id.join(df_agg).drop('tmp', axis=1)
df.head()

purchase_amount_new_sum  authorized_flag_sum  \
card_id         month_lag                                                 
C_ID_4e6213e9bc -11                          17.00                  0.0   
                -10                          11.00                  0.0   
                -9                             NaN                  NaN   
                -8                             NaN                  NaN   
                -7                           17.52                  0.0   

                           installments_sum  purchase_date_count  \
card_id         month_lag                                          
C_ID_4e6213e9bc -11                     2.0                  2.0   
                -10                     1.0                  1.0   
                -9                      NaN                  NaN   
                -8                      NaN                  NaN   
                -7                      3.0                  3.0   

                           category_1_sum  
card_id         month_lag                  
C_ID_4e6213e9bc -11                   0.0  
                -10                   0.0  
                -9                    NaN  
                -8                    NaN  
                -7                    0.0

In [175]:
# move_list = [col for col in df.columns if col.count('amount')]
# move_list += [col for col in df.columns if col.count('install')]
# move_list += [col for col in df.columns if col.count('nunique')]
# move_list += [col for col in df.columns if col.count('count')]
# move_list += [col for col in df.columns if col.count('auth')]
# move_list += [col for col in df.columns if col.count('category')]
# move_list = [col for col in df.columns if col.count('amount')]
aggs = {}
method = ['mean', 'sum'][1]
for col in df.columns:
    aggs[col] = [method]
#     aggs[col] = ['sum']
window_list = [12, 6, 4, 3, 2, 1]
roll_list = []
tmp = df.reset_index().drop('month_lag', axis=1)
for window in window_list:
    df_agg = tmp.groupby(key).rolling(window=window, min_periods=1).mean()
    new_cols = [f"roll_{method}_window{window}_{col}" for col in df_agg.columns]
    df_agg.columns = new_cols
    roll_list.append(df_agg)
    display(roll_list[-1].head())

roll_sum_window12_authorized_flag_sum  \
card_id                                                          
C_ID_00007093c1 1012052                                    0.0   
                1012053                                    0.0   
                1012054                                    0.0   
                1012055                                    0.0   
                1012056                                    0.0   

                         roll_sum_window12_category_1_sum  \
card_id                                                     
C_ID_00007093c1 1012052                              0.00   
                1012053                              0.00   
                1012054                              0.00   
                1012055                              0.25   
                1012056                              0.20   

                         roll_sum_window12_installments_sum  \
card_id                                                       
C_ID_00007093c1 1012052                            3.000000   
                1012053                            3.000000   
                1012054                            3.333333   
                1012055                            5.500000   
                1012056                            4.800000   

                         roll_sum_window12_purchase_amount_new_sum  \
card_id                                                              
C_ID_00007093c1 1012052                                 302.000000   
                1012053                                 366.000000   
                1012054                                 912.043333   
                1012055                                1037.040000   
                1012056                                 867.832000   

                         roll_sum_window12_purchase_date_count  
card_id                                                         
C_ID_00007093c1 1012052                               3.000000  
                1012053                               3.000000  
                1012054                               3.333333  
                1012055                               4.000000  
                1012056                               3.600000

roll_sum_window6_authorized_flag_sum  \
card_id                                                         
C_ID_00007093c1 1012052                                   0.0   
                1012053                                   0.0   
                1012054                                   0.0   
                1012055                                   0.0   
                1012056                                   0.0   

                         roll_sum_window6_category_1_sum  \
card_id                                                    
C_ID_00007093c1 1012052                             0.00   
                1012053                             0.00   
                1012054                             0.00   
                1012055                             0.25   
                1012056                             0.20   

                         roll_sum_window6_installments_sum  \
card_id                                                      
C_ID_00007093c1 1012052                           3.000000   
                1012053                           3.000000   
                1012054                           3.333333   
                1012055                           5.500000   
                1012056                           4.800000   

                         roll_sum_window6_purchase_amount_new_sum  \
card_id                                                             
C_ID_00007093c1 1012052                                302.000000   
                1012053                                366.000000   
                1012054                                912.043333   
                1012055                               1037.040000   
                1012056                                867.832000   

                         roll_sum_window6_purchase_date_count  
card_id                                                        
C_ID_00007093c1 1012052                              3.000000  
                1012053                              3.000000  
                1012054                              3.333333  
                1012055                              4.000000  
                1012056                              3.600000

roll_sum_window4_authorized_flag_sum  \
card_id                                                         
C_ID_00007093c1 1012052                                   0.0   
                1012053                                   0.0   
                1012054                                   0.0   
                1012055                                   0.0   
                1012056                                   0.0   

                         roll_sum_window4_category_1_sum  \
card_id                                                    
C_ID_00007093c1 1012052                             0.00   
                1012053                             0.00   
                1012054                             0.00   
                1012055                             0.25   
                1012056                             0.25   

                         roll_sum_window4_installments_sum  \
card_id                                                      
C_ID_00007093c1 1012052                           3.000000   
                1012053                           3.000000   
                1012054                           3.333333   
                1012055                           5.500000   
                1012056                           5.250000   

                         roll_sum_window4_purchase_amount_new_sum  \
card_id                                                             
C_ID_00007093c1 1012052                                302.000000   
                1012053                                366.000000   
                1012054                                912.043333   
                1012055                               1037.040000   
                1012056                               1009.290000   

                         roll_sum_window4_purchase_date_count  
card_id                                                        
C_ID_00007093c1 1012052                              3.000000  
                1012053                              3.000000  
                1012054                              3.333333  
                1012055                              4.000000  
                1012056                              3.750000

roll_sum_window3_authorized_flag_sum  \
card_id                                                         
C_ID_00007093c1 1012052                                   0.0   
                1012053                                   0.0   
                1012054                                   0.0   
                1012055                                   0.0   
                1012056                                   0.0   

                         roll_sum_window3_category_1_sum  \
card_id                                                    
C_ID_00007093c1 1012052                         0.000000   
                1012053                         0.000000   
                1012054                         0.000000   
                1012055                         0.333333   
                1012056                         0.333333   

                         roll_sum_window3_installments_sum  \
card_id                                                      
C_ID_00007093c1 1012052                           3.000000   
                1012053                           3.000000   
                1012054                           3.333333   
                1012055                           6.333333   
                1012056                           6.000000   

                         roll_sum_window3_purchase_amount_new_sum  \
card_id                                                             
C_ID_00007093c1 1012052                                302.000000   
                1012053                                366.000000   
                1012054                                912.043333   
                1012055                               1282.053333   
                1012056                               1202.386667   

                         roll_sum_window3_purchase_date_count  
card_id                                                        
C_ID_00007093c1 1012052                              3.000000  
                1012053                              3.000000  
                1012054                              3.333333  
                1012055                              4.333333  
                1012056                              4.000000

roll_sum_window2_authorized_flag_sum  \
card_id                                                         
C_ID_00007093c1 1012052                                   0.0   
                1012053                                   0.0   
                1012054                                   0.0   
                1012055                                   0.0   
                1012056                                   0.0   

                         roll_sum_window2_category_1_sum  \
card_id                                                    
C_ID_00007093c1 1012052                              0.0   
                1012053                              0.0   
                1012054                              0.0   
                1012055                              0.5   
                1012056                              0.5   

                         roll_sum_window2_installments_sum  \
card_id                                                      
C_ID_00007093c1 1012052                                3.0   
                1012053                                3.0   
                1012054                                3.5   
                1012055                                8.0   
                1012056                                7.0   

                         roll_sum_window2_purchase_amount_new_sum  \
card_id                                                             
C_ID_00007093c1 1012052                                   302.000   
                1012053                                   366.000   
                1012054                                  1217.065   
                1012055                                  1708.080   
                1012056                                   801.515   

                         roll_sum_window2_purchase_date_count  
card_id                                                        
C_ID_00007093c1 1012052                                   3.0  
                1012053                                   3.0  
                1012054                                   3.5  
                1012055                                   5.0  
                1012056                                   4.0

roll_sum_window1_authorized_flag_sum  \
card_id                                                         
C_ID_00007093c1 1012052                                   0.0   
                1012053                                   0.0   
                1012054                                   0.0   
                1012055                                   0.0   
                1012056                                   0.0   

                         roll_sum_window1_category_1_sum  \
card_id                                                    
C_ID_00007093c1 1012052                              0.0   
                1012053                              0.0   
                1012054                              0.0   
                1012055                              1.0   
                1012056                              0.0   

                         roll_sum_window1_installments_sum  \
card_id                                                      
C_ID_00007093c1 1012052                                3.0   
                1012053                                3.0   
                1012054                                4.0   
                1012055                               12.0   
                1012056                                2.0   

                         roll_sum_window1_purchase_amount_new_sum  \
card_id                                                             
C_ID_00007093c1 1012052                                    302.00   
                1012053                                    430.00   
                1012054                                   2004.13   
                1012055                                   1412.03   
                1012056                                    191.00   

                         roll_sum_window1_purchase_date_count  
card_id                                                        
C_ID_00007093c1 1012052                                   3.0  
                1012053                                   3.0  
                1012054                                   4.0  
                1012055                                   6.0  
                1012056                                   2.0

In [176]:
# window_list = [12, 6, 4, 3, 2, 1]
df_roll = pd.concat(roll_list, axis=1)
new_cols = [f"{fname}_{col}" for col in df_roll.columns]
df_roll.columns = new_cols

df_roll.reset_index(inplace=True)
df_idx = df_roll[[key, 'level_1']].groupby(key).max()
df_idx.reset_index(inplace=True)
df_roll = df_roll.merge(df_idx, how='inner', on=key)
df_roll['month_lag'] = df_roll['level_1_x'] - df_roll['level_1_y']
df_roll.drop(['level_1_x', 'level_1_y'], axis=1, inplace=True)
df_roll.set_index([key, 'month_lag'], inplace=True)
df = df.join(df_roll)
print(df.shape)
df.head(20)

(2060822, 35)


purchase_amount_new_sum  authorized_flag_sum  \
card_id         month_lag                                                 
C_ID_4e6213e9bc -11                          17.00                  0.0   
                -10                          11.00                  0.0   
                -9                             NaN                  NaN   
                -8                             NaN                  NaN   
                -7                           17.52                  0.0   
                -6                             NaN                  NaN   
                -5                           30.78                  0.0   
                -4                             NaN                  NaN   
                -3                             NaN                  NaN   
                -2                             NaN                  NaN   
                -1                             NaN                  NaN   
                 0                           55.00                  0.0   
C_ID_5037ff576e -11                        1443.00                  0.0   
                -10                            NaN                  NaN   
                -9                             NaN                  NaN   
                -8                             NaN                  NaN   
                -7                             NaN                  NaN   
                -6                         4875.80                  0.0   
                -5                         3466.61                  0.0   
                -4                             NaN                  NaN   

                           installments_sum  purchase_date_count  \
card_id         month_lag                                          
C_ID_4e6213e9bc -11                     2.0                  2.0   
                -10                     1.0                  1.0   
                -9                      NaN                  NaN   
                -8                      NaN                  NaN   
                -7                      3.0                  3.0   
                -6                      NaN                  NaN   
                -5                      2.0                  2.0   
                -4                      NaN                  NaN   
                -3                      NaN                  NaN   
                -2                      NaN                  NaN   
                -1                      NaN                  NaN   
                 0                      1.0                  1.0   
C_ID_5037ff576e -11                     5.0                  1.0   
                -10                     NaN                  NaN   
                -9                      NaN                  NaN   
                -8                      NaN                  NaN   
                -7                      NaN                  NaN   
                -6                      4.0                  4.0   
                -5                      3.0                  3.0   
                -4                      NaN                  NaN   

                           category_1_sum  \
card_id         month_lag                   
C_ID_4e6213e9bc -11                   0.0   
                -10                   0.0   
                -9                    NaN   
                -8                    NaN   
                -7                    0.0   
                -6                    NaN   
                -5                    0.0   
                -4                    NaN   
                -3                    NaN   
                -2                    NaN   
                -1                    NaN   
                 0                    0.0   
C_ID_5037ff576e -11                   0.0   
                -10                   NaN   
                -9                    NaN   
                -8                    NaN   
                -7                    NaN   
                -6      

In [177]:
#========================================================================
# save
df.fillna(-1, inplace=True)
utils.to_df_pkl(df=df, path='../input/', fname=f'{fname}_roll_method{method}_future_amount_pred')
#========================================================================

0it [00:00, ?it/s]

shape: (2060822, 35)


3it [00:01,  2.25it/s]


In [3]:
base = utils.read_pkl_gzip('../input/base_type_group.gz')[[key, target]].set_index(key)
path_list = glob.glob('../input/auth*future*')
for path in path_list:
    tmp = utils.read_df_pkl(path)
    display(tmp.head())
    sys.exit()


100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


purchase_amount_new_sum  purchase_amount_new_mean  \
card_id         month_lag                                                      
C_ID_4e6213e9bc -11                          17.00                      8.50   
                -10                          11.00                     11.00   
                -9                           -1.00                     -1.00   
                -8                           -1.00                     -1.00   
                -7                           17.52                      5.84   

                           purchase_amount_new_max  purchase_amount_new_min  \
card_id         month_lag                                                     
C_ID_4e6213e9bc -11                           8.50                     8.50   
                -10                          11.00                    11.00   
                -9                           -1.00                    -1.00   
                -8                           -1.00                    -1.00   
                -7                            5.84                     5.84   

                           purchase_amount_new_std  authorized_flag_sum  \
card_id         month_lag                                                 
C_ID_4e6213e9bc -11                            0.0                  0.0   
                -10                           -1.0                  0.0   
                -9                            -1.0                 -1.0   
                -8                            -1.0                 -1.0   
                -7                             0.0                  0.0   

                           authorized_flag_mean  installments_mean  \
card_id         month_lag                                            
C_ID_4e6213e9bc -11                         0.0                1.0   
                -10                         0.0                1.0   
                -9                         -1.0               -1.0   
                -8                         -1.0               -1.0   
                -7                          0.0                1.0   

                           installments_max  installments_min  \
card_id         month_lag                                       
C_ID_4e6213e9bc -11                     1.0               1.0   
                -10                     1.0               1.0   
                -9                     -1.0              -1.0   
                -8                     -1.0              -1.0   
                -7                      1.0               1.0   

                           yyyymmdd_nunique  purchase_date_count  \
card_id         month_lag                                          
C_ID_4e6213e9bc -11                     1.0                  2.0   
                -10                     1.0                  1.0   
                -9                     -1.0                 -1.0   
                -8                     -1.0                 -1.0   
                -7                      1.0                  3.0   

                           subsector_id_nunique  merchant_category_id_nunique  \
card_id         month_lag                                                       
C_ID_4e6213e9bc -11                         1.0                           1.0   
                -10                         1.0                           1.0   
                -9                         -1.0                          -1.0   
                -8                         -1.0                          -1.0   
                -7                          1.0                           1.0   

                           merchant_id_nunique  state_id_nunique  \
card_id         month_lag                                          
C_ID_4e6213e9bc -11                        1.0               1.0   
                -10                        1.0               1.0   
                -9                        -1.0              -1.0   
                -8                        -1.0        

SystemExit: 